In [1]:
from depmapomics import tracker
track = tracker.getTracker()

In [14]:
track[(track.internal_bam_filepath.isna()) & (track.datatype == 'rna')]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_line_name,participant_id,cellosaurus_id,...,19q3,19q4,20q1,20q2,20q4,20q,21Q1,21Q2,parent_id,21Q3
CDS-ABH0uZ,ACH-001767,1,NaN,NaN,rna,311757376.0,NaN,CCLFPEDS0018T,PT-pmZkEqFb,U,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0


In [4]:
track[(track.low_quality==1)&(track.datatype=='wgs')]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_line_name,participant_id,cellosaurus_id,...,19q3,19q4,20q1,20q2,20q4,20q,21Q1,21Q2,parent_id,21Q3
CDS-dIijHP,ACH-000309,1,NaN,NaN,wgs,7.169431e+10,NaN,SKLU1,PT-FmbFY70A,CVCL_0629,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,NaN,NaN
CDS-ovDo8V,ACH-000890,1,NaN,NaN,wgs,9.124674e+10,NaN,SW1271,PT-a7rG9fcn,CVCL_1716,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,NaN,NaN
CDS-mYOC4j,ACH-000893,1,NaN,NaN,wgs,1.432370e+11,NaN,NCIH1651,PT-LRgbHk7Q,CVCL_1484,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
CDS-Wszh2o,ACH-000860,1,NaN,NaN,wgs,8.011841e+10,NaN,NCIH358,PT-HLoL0NZn,CVCL_1559,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
CDS-3PGQ84,ACH-001553,1,NaN,NaN,wgs,7.667525e+10,NaN,MEGA2,PT-SMrpbPt7,CVCL_1832,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
CDS-AqZLna,ACH-002512,1,NaN,NaN,wgs,8.103403e+10,NaN,M1610113,PT-pUBtKkYR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN


In [1]:
from genepy.utils import helper as h

from taigapy import TaigaClient
from celligner import Celligner
tc = TaigaClient()

#autoreload
%load_ext autoreload
%autoreload 2

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:14: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "l2_norm" failed type inference due to: Invalid use of Function(<function norm at 0x7f9223703158>) with argument(s) of type(s): (axis=Literal[int](1), x=array(float32, 2d, A))
 * parameterized
In definition 0:
    TypeError: norm_impl() got an unexpected keyword argument 'x'
    raised from /home/jeremie/miniconda3/lib/python3.7/site-packages/numba/typing/templates.py:517
In definition 1:
    TypeError: norm_impl() got an unexpected keyword argument 'x'
    raised from /home/jeremie/miniconda3/lib/python3.7/site-packages/numba/typing/t

def compute_correction(data1, data2, mnn1, mnn2, data2_or_raw2, sigma):
^

  warnings.warn(errors.NumbaDeprecationWarning(msg, state.func_ir.loc))
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "adjust_s_variance" failed type inference due to: Untyped global name 'sq_dist_to_line': cannot determine Numba type of <class 'numba.ir.UndefinedType'>

File "../miniconda3/lib/python3.7/site-packages/mnnpy/utils.py", line 209:
def adjust_s_variance(data1, data2, curcell, curvect, sigma):
    <source elided>
        sameproj = np.dot(grad, samecell)
        samedist = sq_dist_to_line(curcell, grad, samecell)
        ^

  @jit(float32(float32[:, :], float32[:, :], float32[:], float32[:], float32), nogil=True)
/home/jeremie/miniconda3/lib/python3.7/site-packages/mnnpy/utils.py:199: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled becaus

In [2]:
# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/

# load  TCGA expression
# this dataset was generated from  ,using this script:
TCGA_expression = tc.get(name='celligner-input-9827',
                         file='tumor_expression')


No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [3]:
# subset gene names to ensembl ids only
CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

common = set(CCLE_expression.columns).intersection(
    set(TCGA_expression.columns))
CCLE_expression = CCLE_expression[list(common)]
TCGA_expression = TCGA_expression[list(common)]


In [17]:
import pandas as pd

In [8]:
from celligner import limma

In [37]:
da = pd.DataFrame(index=TCGA_expression.index[:20], columns=['(Intercept)','C1C','C2C','C3C',"C4C"], data=np.array([
    [1,1,0,0,0],
    [1,1,0,0,0],
    [1,1,0,0,0],
    [1,0,1,0,0],
    [1,0,1,0,0],
    [1,0,1,0,0],
    [1,0,0,1,0],
    [1,0,0,1,0],
    [1,0,0,1,0],
    [1,0,0,1,0],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
    [1,0,0,0,1],
]))

In [50]:
me = limma.limmapy().lmFit(TCGA_expression.iloc[:20],da).eBayes(trend = False).fit

you need to have R installed with the limma library installed
3.2.1


R[write to console]: Error in (function (object, design = NULL, ndups = 1, spacing = 1, block = NULL,  : 
  row dimension of design doesn't match column dimension of data object



RRuntimeError: Error in (function (object, design = NULL, ndups = 1, spacing = 1, block = NULL,  : 
  row dimension of design doesn't match column dimension of data object


In [ ]:
me

In [47]:
list(me.fit)

[array([[-0.        , -0.        ,  0.        , -0.        ,         nan],
        [-0.        , -0.        ,  0.        , -0.        ,         nan],
        [ 2.8       ,  0.53333333, -0.46666667, -1.3       ,         nan],
        ...,
        [ 0.1       ,  1.9       ,  0.9       ,  0.15      ,         nan],
        [-0.        , -0.        ,  0.        , -0.        ,         nan],
        [-0.        , -0.        ,  0.        , -0.        ,         nan]]),
 array([4], dtype=int32),
 <rpy2.rinterface.NULLType object at 0x7f922fa151c8> [RTYPES.NILSXP],
 R object with classes: ('qr',) mapped to:
 [FloatSe..., FloatSe..., IntSexp..., FloatSe..., IntSexp...]
   qr: <class 'rpy2.rinterface.FloatSexpVector'>
   <rpy2.rinterface.FloatSexpVector object at 0x7f9114811dc8> [RTYPES.REALSXP]
   qraux: <class 'rpy2.rinterface.FloatSexpVector'>
   <rpy2.rinterface.FloatSexpVector object at 0x7f9114811b48> [RTYPES.REALSXP]
   pivot: <class 'rpy2.rinterface.IntSexpVector'>
   <rpy2.rinterface.IntSe

In [49]:
list(me.fit)[19]

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
                   nan],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
                   nan],
       [1.50018694e-06, 5.12239331e-01, 5.65741273e-01, 8.75745862e-02,
                   nan],
       ...,
       [8.22940240e-01, 5.40552320e-02, 3.39873874e-01, 8.57575499e-01,
                   nan],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
                   nan],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
                   nan]])

In [ ]:
for k, val in {'coefficients':0, 'lods':20, 'p.value':19, 't':17, 'Amean':9}:
    

In [43]:
dict(zip(me.fit.names, map(list,list(me.fit))))

TypeError: 'NULLType' object is not iterable

In [41]:
me.topTable()

R[write to console]: Error in topTableF(fit, number = number, genelist = genelist, adjust.method = adjust.method,  : 
  F-statistics not found in fit
Calls: <Anonymous> -> topTableF

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: Partial NA coefficients for 29643 probe(s) 

R[write to console]: 2: Zero sample variances detected, have been offset away from zero 

R[write to console]: 3: 
R[write to console]: In .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim,  :
R[write to console]: 
 
R[write to console]:  Estimation of var.prior failed - set to default value



RRuntimeError: Error in topTableF(fit, number = number, genelist = genelist, adjust.method = adjust.method,  : 
  F-statistics not found in fit
Calls: <Anonymous> -> topTableF


In [7]:
my_alligner = Celligner(snn_kwargs={"neighbor_num": 35, "min_shared_neighbor_proportion": 1/15,})

my_alligner.fit(TCGA_expression)

fetching gene names from biomart cache
using only usefull genes
found 29593 common genes
creating a fit dataset..
reducing dimensionality...
clustering...
doing differential expression analysis on the clusters
running differential expression on 127 clusters
> /home/jeremie/celligner/celligner/__init__.py(69)runDiffExprOnCluster()
-> design_matrix = pd.DataFrame(index=expression.index,
(Pdb) n
> /home/jeremie/celligner/celligner/__init__.py(70)runDiffExprOnCluster()
-> data=np.array([clustered==i for i in set(clustered)]).T,
(Pdb) 
> /home/jeremie/celligner/celligner/__init__.py(71)runDiffExprOnCluster()
-> columns=[str(i)+"C" for i in set(clustered)])
(Pdb) 
> /home/jeremie/celligner/celligner/__init__.py(72)runDiffExprOnCluster()
-> design_matrix.index = design_matrix.index.astype(str).str.replace('-', '.')
(Pdb) 
> /home/jeremie/celligner/celligner/__init__.py(73)runDiffExprOnCluster()
-> formula = '~'+'+'.join([str(i)+'C' for i in set(clustered)])
(Pdb) 
> /home/jeremie/celligner/ce

R[write to console]: Error in parse(text = x, keep.source = FALSE) : 
  <text>:1:3: unexpected symbol
1: ~0C
      ^
Calls: <Anonymous> ... formula -> formula.character -> formula -> eval -> parse



rpy2.rinterface_lib.embedded.RRuntimeError: Error in parse(text = x, keep.source = FALSE) : 
  <text>:1:3: unexpected symbol
1: ~0C
      ^
Calls: <Anonymous> ... formula -> formula.character -> formula -> eval -> parse
> /home/jeremie/celligner/celligner/__init__.py(94)runDiffExprOnCluster()
-> design_formula=formula, gene_column="gene_id")
(Pdb) q


BdbQuit: 

In [53]:
hgnc_complete_set = tc.get(
    name='hgnc-87ab', version=5, file='hgnc_complete_set')

ACH = tc.get(name='arxspan-cell-line-export-f808', version=752, file='ACH')

CCLE_annotations = tc.get(name='celligner-input-9827',
                          version=1, file='CCLE_annotations')

tumor_annotations = tc.get(name='celligner-input-9827',
                           version=1, file='tumor_annotations')
                           
tumor_expression = tc.get(name='celligner-input-9827',
                          version=1, file='tumor_expression')
